# Data
#### We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

The data is stored in the ./data/ directory

In [182]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [144]:
headers = ["BI-RADS", "Age", "Shape", "Margin", "Density", "Severity"]
dataset = pd.read_csv('./data/mammographic_masses.data', header=None, names=headers, na_values='?')
dataset.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [145]:
dataset.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [146]:
dataset.isna().sum()

BI-RADS      2
Age          5
Shape       31
Margin      48
Density     76
Severity     0
dtype: int64

In [147]:
clean.corr()

,BI-RADS,Age,Shape,Margin,Density,Severity
BI-RADS,1.000000,0.094623,0.180120,0.157771,0.028356,0.223826
Age,0.094623,1.000000,0.380096,0.420913,0.052417,0.455216
Shape,0.180120,0.380096,1.000000,0.738014,0.073969,0.564763
Margin,0.157771,0.420913,0.738014,1.000000,0.124860,0.574498
Density,0.028356,0.052417,0.073969,0.124860,1.000000,0.068651
Severity,0.223826,0.455216,0.564763,0.574498,0.068651,1.000000


This is a binary classification problem, our goal is to correctly predict the severity of a cancer based on the other features. 

The data work-flow is inspired by this video: https://www.youtube.com/watch?v=MpFZUshKypk

In [93]:
def stage_details(f):
    def wrapper(dataframe, *args, **kwargs):
        result = f(dataframe, *args, **kwargs)
        print(f'<{f.__name__}> dataframe shape: {result.shape}')
        return result
    
    return wrapper

In [209]:
@stage_details
def start_pipeline(dataframe):
    return dataframe.copy()

@stage_details
def format_data(dataframe):
    dataframe = dataframe.drop(columns=["BI-RADS"])
    return dataframe

@stage_details
def clean_data(dataframe):
    imp = IterativeImputer(max_iter=10, random_state=0)
    clean = dataframe.dropna()
    imp.fit(clean)
    imputed = np.round(imp.transform(dataframe))
    
    imputed_dataframe = pd.DataFrame(data=imputed, columns=["Age", "Shape", "Margin", "Density", "Severity"])
    return imputed_dataframe

@stage_details
def normalise_data(dataframe):
    scaler = StandardScaler()
    
    labels = dataframe['Severity']
    dataframe = dataframe.drop(columns=['Severity'])
    
    scaler.fit(dataframe)
    normalised_data = scaler.transform(dataframe)
    normalised_dataframe = pd.DataFrame(data=normalised_data, columns=["Age", "Shape", "Margin", "Density"])
    normalised_dataframe['Severity'] = labels
    return normalised_dataframe

@stage_details
def shuffle_data(dataframe):
    return prepared_data.sample(frac=1)

@stage_details
def visualise_data(dataframe):
    sns.pairplot(dataframe, hue='Severity', height=2.5)
    return dataframe


In [252]:
prepared_data = (dataset.pipe(start_pipeline)
        .pipe(format_data)
        .pipe(clean_data)
        .pipe(normalise_data)
        .pipe(shuffle_data))

X = prepared_data.drop(columns=['Severity'])
Y = prepared_data['Severity']

<start_pipeline> dataframe shape: (961, 6)
<format_data> dataframe shape: (961, 5)
<clean_data> dataframe shape: (961, 5)
<normalise_data> dataframe shape: (961, 5)
<shuffle_data> dataframe shape: (961, 5)


# Approach \#1, Decision Tree

In [315]:
TrainX, TestX, TrainY, TestY = train_test_split(X, Y, train_size=0.25, shuffle=True)

print(f'Original: X: {X.shape}, Y: {Y.shape}')
print(f'Train: X: {TrainX.shape}, Y: {TrainY.shape}.')
print(f'Test: X: {TestX.shape}, Y: {TestY.shape}.')

Original: X: (961, 4), Y: (961,)
Train: X: (240, 4), Y: (240,).
Test: X: (721, 4), Y: (721,).


In [316]:
from sklearn.tree import DecisionTreeClassifier

In [318]:
Tree = DecisionTreeClassifier()
s = Tree.fit(TrainX, TrainY)


In [322]:
s.score(TestX, TestY)

0.6823855755894591

In [321]:
correct / total

1.0